<a href="https://colab.research.google.com/github/bogard75/untitled/blob/master/rnf_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tf_slim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 352 kB 8.0 MB/s 


In [18]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

#import tensorflow.contrib.slim as slim
#!pip install tf_slim
import tf_slim as slim
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


Instructions for updating:
non-resource variables are not supported in the long term


밴딧

In [19]:
bandit_arms = [0.2, 0, -0.2, -2]
num_arms = len(bandit_arms)

def pull_bandit(bandit):
    #랜덤값 구해서
    result = np.random.randn(1)
    
    if result > bandit:
        # 양의 보상
        return 1
    else:
        # 음의 보상
        return -1

에이전트

In [29]:
tf.get_default_graph()
weights = tf.Variable(tf.ones([num_arms]))
output = tf.nn.softmax(weights)
#print(weights.shape)
reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)

responsible_output = tf.slice(output, action_holder, [1])
loss = -(tf.log(responsible_output)*reward_holder)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
update = optimizer.minimize(loss)



(1,)


학습시키기

In [24]:
total_episodes = 10000

total_reward = np.zeros(num_arms)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i = 0 
    while i < total_episodes:
        actions = sess.run(output)
        a = np.random.choice(actions, p=actions)
        action = np.argmax(actions == a)

        reward = pull_bandit(bandit_arms[action])

        _, resp, ww = sess.run([update, responsible_output, weights], \
                               feed_dict={reward_holder:[reward], action_holder:[action]})
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the " + str(num_arms) + " arms of the bandit: " + str(total_reward))
        i+=1


print("\nThe agent thisks arm " + str(np.argmax(ww)+1) + " is the most promising .... ")
if np.argmax(ww) == np.argmax(-np.array(bandit_arms)):
    print("...and it was right!")
else:
    print("...and it was wrong!")


Running reward for the 4 arms of the bandit: [-1.  0.  0.  0.]
Running reward for the 4 arms of the bandit: [-6.  3.  4. 14.]
Running reward for the 4 arms of the bandit: [-9.  1. 14. 25.]
Running reward for the 4 arms of the bandit: [-9.  2. 16. 38.]
Running reward for the 4 arms of the bandit: [-6.  7. 15. 53.]
Running reward for the 4 arms of the bandit: [-4.  4. 20. 67.]
Running reward for the 4 arms of the bandit: [-6.  4. 18. 75.]
Running reward for the 4 arms of the bandit: [-7.  7. 20. 87.]
Running reward for the 4 arms of the bandit: [-9.  1. 16. 99.]
Running reward for the 4 arms of the bandit: [ -8.   3.  16. 118.]
Running reward for the 4 arms of the bandit: [ -6.   9.  21. 139.]
Running reward for the 4 arms of the bandit: [ -6.  11.  21. 151.]
Running reward for the 4 arms of the bandit: [-11.  14.  23. 169.]
Running reward for the 4 arms of the bandit: [ -9.  14.  21. 181.]
Running reward for the 4 arms of the bandit: [-14.  10.  19. 200.]
Running reward for the 4 arms o